# Lab: Used Car Data Analysis
---
In this lab, we will use the UsedCars.csv to practice ETL operations on data in Azure Databricks, create a dashboard and execute a simple ML regression on our data. This will all be done by using our Containers in the Blob Storage and through the Secrets CLI

## Connect to the Storage

### 1. Mount a new Blob Container  
The container should be have one file uploaded to it. The file is included in the Workshop folder and it is called: UsedCars.csv.  
  
Fill in the values that are surrounded by `<>`.  
- Enter the name of the Storage Account where it says `< YOUR STORAGE ACCOUNT NAME >`  
  - If your storage account name is `datastore` then the code should be `storage_account = 'datastore'`  
- Enter the name of the container in the Storage Account where it says `< YOUR CONTAINER NAME >`  
  - If the name of the contianer is `mydata` then the code should say `container = 'mydata'`  
- Enter the SAS token that was generated for this Demo where it sas `< YOUR SAS TOKEN >`  
  - If the value of the SAS token is `3ij983jf3j02jtgeo-0fk23jf048fhwf` than the code should say `blobKey = '3ij983jf3j02jtgeo-0fk23jf048fhwf'`

In [0]:
# DO NOT EDIT 
# The below function will handle mounting for us. We will call it in the next cell.
def mountblobstorage(storage_account, container, blobKey, mnt_location, blobEndpoint):
  try:
    dbutils.fs.mount(
      source = blobEndpoint,
      mount_point = mnt_location,
      extra_configs = {"fs.azure.sas.{1}.{0}.blob.core.windows.net".format(storage_account, container):blobKey})
  except Exception as e:
    print(e)
    dbutils.fs.unmount(mnt_location)
    mountblobstorage(storage_account, container, blobKey, mnt_location, blobEndpoint)

### Wait a second... are you really pasting your credentials here without secrets management?

Copying and pasting tokens and keys directly into Azure Databricks poses security risks and challenges. It's preferable to use **Azure Key Vault** or the **the creation of secrets through the CLI**. Storing credentials in code or notebooks increases the risk of accidental exposure. 

#### Azure Key Vault
Azure Key Vault provides a secure storage for secrets, with robust access control and permission management. It allows centralized management, enabling easy updates and rotation of secrets without code changes. Key Vault offers auditing capabilities for compliance and integrates seamlessly with Azure Databricks. Utilizing Key Vault enhances security, access control, management, auditing, and flexibility in handling secrets.

#### Secrets through the CLI
The creation of secrets through the CLI offers a standardized and programmatic way to securely retrieve secrets during runtime, eliminating the need for hardcoded or exposed sensitive information. It enables integration with different development frameworks, enhancing versatility. By utilizing the Secrets API, you can benefit from advanced access control, auditing, and compliance features provided by the secrets store. This approach promotes best practices in security, simplifies secrets management, and ensures the protection of sensitive data in your applications.

First, we will need to install or update the CLI in our local machines. Please follow the guide here [Databricks CLI](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/cli/)

Secondly, we will need to create the secrets in our CLI. Follow the guide here [Secrets CLI](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/cli/secrets-cli)

In [0]:
dbutils.fs.unmount("/mnt/usedcars")

# FILL IN the below values denoted with <>
storage_account = "< YOUR STORAGE ACCOUNT NAME >"
container = "< YOUR CONTAINER NAME >"
blobKey = "< YOUR SAS TOKEN >"
	
mnt_location = "/mnt/usedcars"
blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)
mountblobstorage(storage_account, container, blobKey, mnt_location, blobEndpoint)

### Step 2: Check the contents of the mounted storage  
Use the `dbutils.fs` utility to check the contents of the newly mounted Blob Container.

In [0]:
dbutils.fs.ls("/mnt/usedcars")

### Step 3: Take a Look at the Data  
We can take a look at the contents of the csv file by invoking head.

In [0]:
%fs head "/mnt/usedcars/UsedCars.csv"

## Structure our Data  
A best practice is to explicitly define a schema when loading our data into a dataframe. This improves our execution time by reducing the number of jobs submitted to our cluster.  
Fill in the missing code where it says `<FILL IN>`.

In [0]:
#FILL IN
from pyspark.sql.types import *

UsedCarSchema = StructType([StructField("Index", IntegerType(), True),
                     StructField("Price", FloatType(), True),
                     StructField("Age", FloatType(), True),
                     StructField("KM", FloatType(), True),
                     StructField("FuelType", StringType(), True),
                     StructField("HP", IntegerType(), True),
                     StructField("MetColor", IntegerType(), True),
                     StructField("Automatic", IntegerType(), True),
                     StructField("CC", IntegerType(), True),
                     StructField("Doors", IntegerType(), True),
                     "<FILL IN>"
                     ])
                     

###Compare the Execution Times between `inferSchema` and Passing a `StructType` schema object  

When working with structured data in Apache Spark, you have two options for defining the schema of your data: using inferSchema or passing a StructType schema object. Let's compare the execution times of these two approaches:

- **inferSchema**: The inferSchema method automatically infers the schema of the data by sampling a portion of the dataset. It reads the data and analyzes the values in each column to determine the data types. While this approach is convenient and requires minimal code, it can be slower for large datasets as it needs to read and analyze a sample of the data. The execution time for inferSchema depends on the size of the sample and the complexity of the data.

- **StructType schema object**: Alternatively, you can explicitly define the schema using a StructType object. This approach requires you to specify the data types and structure of each column in your dataset. While it involves writing more code upfront, it provides precise control over the schema and can be faster for large datasets. The execution time for passing a StructType schema object is typically faster as Spark does not need to infer the schema by analyzing the data.

Take a look at the execution time for our two implementations.

In [0]:
%timeit df = spark.read.format("csv").schema(UsedCarSchema).option("header", True).load("/mnt/usedcars/UsedCars.csv")

In [0]:
%timeit dfInferred = spark.read.format("csv").option("inferschema", True).option("header", True).load("/mnt/usedcars/UsedCars.csv")

By explicitly defining our schema, we were able to improve performance by reducing number of jobs submitted to our cluster.

## Data Transformation
### Begin Transforming DataFrame  
Let's start transforming our DataFrame. First, we need to read in the data with the schema object defined above.  
Fill in the missing code where it says `<FILL IN>`.

In [0]:
#FILL IN
df = "<FILL IN>"
display(df)

As we may see on the table above, databricks already has a built-in `Index` so we don't need to have a column with that information. Let's go ahead and drop that column.

In [0]:
df = df.drop("Index")

### Eliminate Duplicates  
We need to clean up our `FuelType` column to eliminate duplicate values.
Let's start by taking a look at all the fuel types that we have on our dataset

In [0]:
#FILL IN
display(df.select("FuelType")."<FILL IN>")

If cars are your specialty, you might know that Compressed natural gas can also be called "methane". At the same time, in our dataset, we can see that we have the abbreviation "CNG". 

Let's do some data cleaning:

In [0]:
df = df.na.replace(["Diesel", "Petrol", "CNG", "methane", "CompressedNaturalGas"], ["diesel", "petrol", "cng", "cng", "cng"], "FuelType")
display(df)

In this case, we are only handling the duplicate values for `FuelType`, however, it's a good practice to analyze all columns in your dataset

### Handling `null` values  
There mht be null values represented in our dataset. It's important to remove null values as they can hinder the accuracy and reliability of data analysis and machine learning models.






If there are any, let's drop any rows containing null values.

**Hint**: [Stackoverflow](https://stackoverflow.com/questions/61499910/alias-in-pyspark) is always our friend

In [0]:
#First let's check the null values
"<FILL IN>"

In [0]:
df = df.na.drop()

### Global Managed Table  

In Databricks, a global managed table is a type of table that is stored and managed in a global metastore, allowing it to be accessed across different Databricks workspaces and clusters within an Azure Databricks account.
When a table is created as a global managed table, its metadata and schema are stored in the global metastore, while the actual data resides in a distributed file system, such as Azure Data Lake Storage. This enables seamless sharing and collaboration across different workspaces and clusters within the same Databricks account.





As we have finished our data cleaning, and we want the rest of our company to have access to this beautifully cleaned table, let's go ahead and create a global managed table from the DataFrame.

In [0]:
df.write.mode("overwrite").saveAsTable("usedcars_cleaned")

## Data Exploration 
Explore the data writing Spark SQL queries on our newly created table

### Query 1

In [0]:
%sql
select * from usedcars_cleaned

### Query 2
Introduce a new table containing data for transmission type. We will join it with usedcars_cleaned table

In [0]:
%sql create table if not exists car_transmission (transmission_id int, transmission_type string)

In [0]:
%sql
insert overwrite table car_transmission values (0, "Manual"), (1, "Automatic");

In [0]:
%sql select * from car_transmission

### Query 3

Let's check the number of cars that have each transmission type

In [0]:
%sql
select "<FILL IN>", count("<FILL IN>") as cnt
from usedcars_cleaned join car_transmission on usedcars_cleaned.Automatic = car_transmission.transmission_id
group by transmission_type

## Dashboard
The following queries produce some interesting insights into the data. Try Databricks' visuals, and output the results in a bar or pie chart. Get to know more on [Visualization in Databricks](https://learn.microsoft.com/en-us/azure/databricks/visualizations/)

### Number of Cars Available
We firstly need to analyze how many cars we have based on its `FuelType`. We will summarize this information on a pie chart

In [0]:
%sql
select "<FILL IN>", count("<FILL IN>")
from usedcars_cleaned
group by "<FILL IN>"

### Car Prices based on Fuel Type
Our company needs to have information on the average price of the cars based on their `FuelType`. Let's make a bar chart based on that!

In [0]:
%sql
select "<FILL IN>", avg(Price) as AvgPrice
from usedcars_cleaned
group by FuelType

### Cars Longevity
What type of cars might we be selling? Do we have older cars (or the so called *classics*) or do we have some new-second-hand cars? Let's have a look at that, in a line chart.

In [0]:
%sql
select Age, count("<FILL IN>")
from usedcars_cleaned
group by Age

### Kilometers Driven

We might think there is a connection between the number of km driven in a car with it's average price. Let's check that on a bar chart

In [0]:
%sql
select t.range as km_driven, avg(Price) as avgPrice
from (select case  
    when KM between 0 and 49999 then '0 < x < 50k'
    when KM between 50000 and 99999 then '50k <= x < 100k'
    else 'x >= 100k' end as range, Price
  from usedcars_cleaned) t
group by t.range
order by km_driven

### Price distribution

When we are looking at the price of our cars, let's see how they are distributed in a graph

In [0]:
%sql
select Price, count("<FILL IN>")
from usedcars_cleaned
group by "<FILL IN>"

### DYI Query

Feel free to imagine something you would want to know! Try writing a Spark SQL query of your own to explore the data set

In [0]:
%sql
-- "<FILL IN>"

Now it's time to... **create your own dashboard**! Using the knowledge you acquired on the 5th lab of this workshop, let's create a dashboard with these 5 visualizations that we have created. Adapt it however you want, and in the end take a printscreen and share with the rest of the class

## Machine Learning 

It's time to start our machine learning project aimed at predicting the price of used cars. With a focus on providing accurate and reliable pricing estimates, our company is leveraging a cleaned dataset containing valuable information on various attributes of used cars. 

By utilizing this dataset, we aim to build a robust machine learning pipeline that can effectively analyze the features of a used car and **provide an estimated price** based on historical data. This project will help to assist our sellers in making informed decisions (for example, on the pricing of a *new* car added to our website) by offering insights into the fair market value of used cars, thereby enhancing transparency and facilitating fair transactions in the automotive marketplace.

The normal steps of a machine learning project are:

1. **Data Cleaning** - We've already worked on this by cleaning duplicate values and making sure there aren't any null values
2. **Exploratory Data Analysis (EDA)** - We've already seen our data, visualizing most of the important feature, identifying patterns and some important correlations.
3. **Feature Engineering** - We are going to start here! In order to do this, we should select the features (columns) that bring the most value to predict the price of a car. The simplest way we can do this, is by keeping only the variables that have continuous values (integer or floats)
4. **Split our data** - dividing a dataset into separate training and testing subsets, allowing the machine learning model to be trained on the training set and evaluated on the testing set to assess its performance on unseen data.
5. **Model Selection** - We are going to do a simple Regression
6. **Model Evaluation** - Let's finish by checking the performance of our models

### Feature Engineering
Let's remove the columns that are not of the type string and have continuous values: Price, Age, KM and the Weight of the Car

**Hint**: On the first fill in you will need to use the table that already has a schema defined

In [0]:
non_string_columns = [column for column in df.columns if "<FILL IN>".schema[column].dataType != StringType()]
df_filtered = "<FILL IN>"

display(df_filtered)

### Split our data
Now we have the data ready for the prediction, let's separate our dataset to put it to work. In the cell below you won't have any action required. We will separate our dataset first into the *X* dataset, that will have every column but the price, and the *y* that will have the information on the price. 

Then we will follow onto separating our dataset into 2 subsets:
1. The **training set** is used to teach the machine learning model how to make predictions based on the other columns. It will help the model learn the underlying patterns and relationships in the data.

2. The **testing set** is used to assess how well the trained model performs on new, unseen data. It helps measure the model's accuracy and see how effectively it can predict or classify the target variable for data it has not seen before.

In [0]:
# Separate the input features (X) and the target variable (y)
X = "<FILL IN>"  # DataFrame with input features
y = "<FILL IN>"  # DataFrame with target variable

# Split the data into training and testing sets
X_train, X_test = X.randomSplit([0.7, 0.3], seed=12345)
y_train, y_test = y.randomSplit([0.7, 0.3], seed=12345)

### Model Selection

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Separate the input features (X) and the target variable (y)
X = df_filtered.drop('price')  # DataFrame with input features
y = df_filtered.select('price')  # DataFrame with target variable

# Create a VectorAssembler to assemble the input features into a single vector column
assembler = VectorAssembler(inputCols=X.columns, outputCol='features')
X_assembled = assembler.transform(X)

# Add the target variable column to the assembled data
assembled_data = X_assembled.join(y)

# Split the data into training and testing sets
X_train, X_test = assembled_data.randomSplit([0.7, 0.3], seed=12345)

# Create a Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol='price', regParam=0.0)

# Fit the model to the training data
lr_model = lr.fit(X_train)

### Model Evaluation

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
predictions = lr_model.transform(X_test)

print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

# Use the model to make predictions on the testing data
predictions = lr_model.transform(X_test)

predictions.show()  